In [45]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import math
from tqdm import tqdm
import numpy as np



def process_date_time(this_df):
    if 'pubtime' in this_df.columns:
        this_df['dt'] = pd.to_datetime(this_df['pubtime'], utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')
    if 'timestamp' in this_df.columns:
        this_df['dt'] = pd.to_datetime(this_df['timestamp'], utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')
    if 'pubtime' in this_df.columns:
        print('process test data date')
        this_df['pub_date'] = this_df['dt'].dt.date
        this_df['pub_date'] = this_df['pub_date'].astype('str')
        lbe = LabelEncoder()
        this_df['pub_date'] = lbe.fit_transform(this_df['pub_date'])
        del this_df['pubtime']
        del this_df['dt']
    if 'timestamp' in this_df.columns:
        print('process train data date')
#         this_df['inter_day'] = this_df['dt'].dt.day
#         this_df['inter_day'].fillna(0, inplace=True)
#         this_df[['inter_day']] = this_df[['inter_day']].astype(int)
#         lbe = LabelEncoder()
#         this_df['inter_day'] = lbe.fit_transform(this_df['inter_day'])
#         this_df['inter_hour'] = round(this_df['dt'].dt.hour)
#         this_df['inter_hour'].fillna(0, inplace=True)
#         this_df[['inter_hour']] = this_df[['inter_hour']].astype(int)
#         lbe = LabelEncoder()
#         this_df['inter_hour'] = lbe.fit_transform(this_df['inter_hour'])
        this_df['hour'] = round(this_df['dt'].dt.hour/2)
        hour_mean=round(this_df['hour'].mean())
        print('hour_mean:',hour_mean)
        this_df['hour'].fillna(hour_mean, inplace=True)
    
        del this_df['timestamp']
        del this_df['dt']


In [46]:
doc_info = pd.read_table('../../data/news_info_new_id.txt',header=None)
doc_info.columns = ['docid', 'title', 'pubtime', 'picnum', 'category1st', 'category2nd', 'keyword']
'''
预处理文档信息数据
:param filedir:
:return:
'''
print('begin process news data')


def clean_picnum(x):
#     print(x)
    # 对picnum数据进行清洗
    if x in ['上海', '云南', '山东', 'NoneType', ''] or x is None or math.isnan(x):
        return 0
    else:
        return int(float(x))
    
doc_info['picnum'] = doc_info['picnum'].apply(lambda x: clean_picnum(x))

process_date_time(doc_info)

for col in tqdm(['category1st', 'category2nd']):
    lbe = LabelEncoder()
    doc_info[col] = doc_info[col].fillna('NAN')
    doc_info[col] = lbe.fit_transform(doc_info[col])

if 'title' in doc_info.columns:
    del doc_info['title']

doc_info['docid'] = doc_info['docid'].astype('int')

print('create word book......')
words = [[], [], [], [], []]

for i in tqdm(doc_info['keyword']):
    key_one = {}
    try:
        splts = i.split(',')
        for sp in splts:
            split_ = sp.split(':')
            try:
                score = float(split_[1])
                key_one[split_[0]] = score
            except:
                continue
        key_one = sorted(key_one.items(), key=lambda x: x[1], reverse=True)
    except:
        key_one = {}
    for j in range(5):
        try:
            key_ = key_one[j][0]
            words[j].append(key_)
        except:
            words[j].append('_#_#_#_')
        
keywords_all = ['_#_#_#_']
for i in range(5):
    keywords_all += words[i]

keywords_all = list(set(keywords_all))
keyword2id = dict(zip(keywords_all, range(1, len(keywords_all) + 1)))

print('generate new key words representation......')
for i in range(5):
    for j in tqdm(range(len(words[i]))):
        words[i][j] = keyword2id[words[i][j]]

doc_info['keyword0'] = words[0]
doc_info['keyword1'] = words[1]
doc_info['keyword2'] = words[2]
doc_info['keyword3'] = words[3]
doc_info['keyword4'] = words[4]

doc_info_key_word_list = list(doc_info['keyword0']) + list(doc_info['keyword1']) + list(doc_info['keyword2']) \
                   + list(doc_info['keyword3']) + list(doc_info['keyword4'])
doc_keyword_id = list(set(doc_info_key_word_list))
new_doc_keyword_id = range(1, len(doc_keyword_id) + 1)
doc_id_dict = dict(zip(doc_keyword_id, new_doc_keyword_id))
# print(len(doc_keyword_id))
# 重新设置文档的关键词id
doc_info['keyword0'] = doc_info['keyword0'].map(doc_id_dict)
doc_info['keyword1'] = doc_info['keyword1'].map(doc_id_dict)
doc_info['keyword2'] = doc_info['keyword2'].map(doc_id_dict)
doc_info['keyword3'] = doc_info['keyword3'].map(doc_id_dict)
doc_info['keyword4'] = doc_info['keyword4'].map(doc_id_dict)

if 'keyword' in doc_info.columns:
    del doc_info['keyword']
doc_info.to_pickle('../../data/wj/doc.pkl')

begin process news data
process test data date


100%|██████████| 2/2 [00:00<00:00,  7.46it/s]


create word book......


100%|██████████| 347466/347466 [00:12<00:00, 26811.37it/s]


generate new key words representation......


100%|██████████| 347466/347466 [00:00<00:00, 746477.58it/s]


In [47]:
user_info = pd.read_table('../../data/user_info_new_id.txt',header=None)
user_info.columns = ['userid', 'device', 'os', 'province', 'city', 'age', 'gender']
'''
    预处理用户数据
    :param filedir:
    :return:
'''
def get_cate(x):
    if type(x) == float:
        return x
    li = x.split(',')
    res = list()
    for i in li:
        lbl, prob = i.split(':')
        res.append([lbl, float(prob)])
    res = sorted(res, key=lambda x: x[1])
    return res[-1][0]

user_info['age'] = user_info['age'].apply(lambda x: get_cate(x))
user_info['gender'] = user_info['gender'].apply(lambda x: get_cate(x))

# label encoding 数字化
for col in tqdm(
        ['device', 'os', 'province', 'city', 'age', 'gender']):
    lbe = LabelEncoder()
    user_info[col] = user_info[col].fillna('NAN')
    user_info[col] = lbe.fit_transform(user_info[col])

print('processed user info')
user_info.to_pickle('../../data/wj/user.pkl')

100%|██████████| 6/6 [00:02<00:00,  2.17it/s]

processed user info


In [48]:
test_info = pd.read_table('../../data/test_info_new_id.txt',header=None)
test_info.columns = ['id', 'userid', 'docid', 'timestamp', 'network', 'refresh']
'''
处理测试集数据
:param filename:
:return:
'''

print('begin process test data')
process_date_time(test_info)
print('done!the total test number is {number}'.format(number=test_info.shape[0]))
test_info.to_pickle('../../data/wj/test.pkl')

begin process test data
process train data date
hour_mean: 7
done!the total test number is 50000


In [ ]:
train_info = pd.read_table('../../data/train_info_new_id.txt',header=None)
train_info.columns = ['userid', 'docid', 'timestamp', 'network', 'refresh', 'position', 'click', 'duration']

In [49]:

'''
处理训练集数据
:param filename:
:return:
'''
print('begin process train data')

train_info['timestamp'].fillna(1624680000000, inplace=True)

process_date_time(train_info)
train_info.to_pickle('../../data/wj/train.pkl')


begin process train data
process train data date
hour_mean: 7
